# Displaying strike and dip symbols on a map using folium
### O. Kaufmann, 2018.

In [5]:
%matplotlib inline

In [6]:
import matplotlib.pyplot as plt
import mplleaflet as mpll
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point 

ModuleNotFoundError: No module named 'mplleaflet'

<div class="alert alert-block alert-danger">
**First start a simple http server to serve the symbols**  
Open a new terminal move to the work directory and run the following command  
`python3 -m http.server 3333 --bind 127.0.0.1`
</div>

### Create a pandas dataframe with the position of the symbols given in the projected coordinate reference system of the map and the strike and dip angles given in degrees using the rigth hand side rule

In [ ]:
df = pd.DataFrame([[120150, 122145,120, 30], 
                   [121450, 122255, 110, 40], 
                   [120874, 123059, 0, 60],
                   [121400, 123500, 245, 10]], 
                  columns=['x', 'y', 'strike', 'dip'])

In [ ]:
gs = gpd.GeoSeries(df.apply(lambda row: Point(row['x'], row['y']), 1),crs={'init': 'epsg:31370'})

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=gs)
gdf

### Create a function to draw a symbol given the position and angles and save this symbol as a png image

In [ ]:
def plot_strike_dip(pos, strike, dip, degrees=True, dip_length=False, linewidth=20., textsize=20.,
                    color='black', png_fig=None):
    # right hand side rule
    import numpy as np
    
    if degrees:
        strike = strike/180*np.pi
        dip = dip/180*np.pi
    
    if dip_length:
        dl = np.sin(dip)
    else:
        dl = 0.4
        
    fig, ax=plt.subplots(figsize=(4,4));
    ax.axis('equal');
    
    ax.plot([-np.cos(np.pi/2.-strike), np.cos(np.pi/2.-strike)], 
            [-np.sin(np.pi/2.-strike), np.sin(np.pi/2.-strike)],
            linewidth=linewidth, color=color);
    ax.plot([0.0, np.sin(strike+np.pi/2)*dl], 
            [0.0, np.cos(strike+np.pi/2)*dl], '-k',
           linewidth=linewidth, color=color);
    
    txt = ax.text(0., 0, '%02.f°' %(dip*180/np.pi), size=textsize, color=color,
                  horizontalalignment='center', verticalalignment='center');
    
    txt.set_rotation(90-strike*180/np.pi);
    txt.set_position((2.5*np.sin(strike+np.pi/2)*dl, 2.5*np.cos(strike+np.pi/2)*dl));
    
    #ax.plot(0., 0., '.c', markersize = 50)
    
    if png_fig is not None:
        ax.axis('off');
        ax.set_position([-1, -1, 1, 1]);
        ax.set_xlim([-1.25,1.25]);
        ax.set_ylim([-1.25,1.25]);
        ax.set_aspect('equal');
        ax.figure.canvas.draw();
        ax.margins(1.);
        ax.tick_params(which='both', direction='in');
        extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted());
        plt.savefig(png_fig + '.png', dpi=166, bbox_inches=extent, format='png', transparent=True);
        ax.figure.clf()

In [ ]:
gdf.apply(lambda row: plot_strike_dip(Point(row['x'], row['y']), row['strike'], row['dip'], textsize=80, png_fig=str(row.name)),1)
for idx, row in gdf.iterrows():
    gdf.loc[idx, 'icon'] = 'http://localhost:3333/' + str(idx) + '.png'
gdf

In [ ]:
import folium
from folium.plugins import FloatImage

In [ ]:
gdf.to_crs({'init': 'epsg:4326'}, inplace=True)

In [ ]:
center=(gdf.loc[0, 'geometry'].y, gdf.loc[0, 'geometry'].x)
center

In [ ]:
from IPython.display import Image
m = folium.Map(center, zoom_start=13)
for idx, row in gdf.iterrows():
        icon_url = row['icon']
        popup = '%03.f° - %02.f°' %(row['strike'], row['dip'])
        icon = folium.features.CustomIcon(icon_url, icon_size=(32, 32))
        marker = folium.Marker([row.geometry.y, row.geometry.x], icon=icon, popup=popup)
        m.add_child(marker)

In [ ]:
m